In [5]:
#dutcomm register offsets
dutcomm_START       = 0x00
dutcomm_STATUS      = 0x04
dutcomm_EXP_OUT_LEN = 0x0c
##########################
#config parameters
EXPECTED_OUT_LEN    = 4  # in 32 bit words
TV_LEN              = 12 # in 32 bit words
NUM_TRACES          = 10
##########################
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import Overlay
import sys
import time
##########################
ol = Overlay("ctrl_top.bit")
xlnk = Xlnk()
dma = ol.axi_dma_0

In [6]:
#configure dutcomm expected_output_lenght
dutcomm = ol.dutcomm_0
dutcomm.write(dutcomm_EXP_OUT_LEN,4)

In [7]:
#test vector
tv = [0x00c00010, 0x59390499, 0x79b065ad, 0x57f2548b, 0x2b539aa1, 
      0x00c10010, 0x11352d06, 0xd4fc0fb3, 0xc0fda1b8, 0x94d8aa1f,
      0x00810010, 0x00800001]

In [8]:
tic = time.time()
for trace in range(NUM_TRACES):
    #prapare buffers
    input_buffer = xlnk.cma_array(shape=(TV_LEN,), dtype=np.uint32)
    output_buffer = xlnk.cma_array(shape=(EXPECTED_OUT_LEN,), dtype=np.uint32)

    for i in range(0, TV_LEN):
       input_buffer[i] = tv[i]
    
    dma.recvchannel.transfer(output_buffer) #configure dma to receive
    dma.sendchannel.transfer(input_buffer)  #configure dma to send 
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    ##display result
    for i in range(4):
       sys.stdout.write(format(output_buffer[i], '08x'))
    sys.stdout.write('\n')
    
    input_buffer.freebuffer()
    output_buffer.freebuffer()
    
toc = time.time()
print("Time elapsed = {} seconds.".format(toc - tic))

5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
5ec6aec2dbf2760a15eaa9d415cf4cee
Time elapsed = 0.05469989776611328 seconds.
